In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_csv('final_data.csv')

In [3]:
#Choose relevant columns
#get dummy data
#train test split
#multiple linear regression
#lasso regression
#random forest
#xgboost
#Adaboost
#tune hyperparameters
#test ensembles

In [4]:
#Choose relevant columns
df.columns

Index(['Superficie', 'Rooms', 'Bedrooms', 'Price', 'Location', 'description',
       'Loc_Paris', 'm2', 'nb_bat', 'nb_good', 'nb_bain', 'Cave', 'balc_terr',
       'Chauffage', 'Rénové', 'Parking', 'Chauffage_Central',
       'Chauffage_Individuel', 'Price_int', 'Last_etage'],
      dtype='object')

In [5]:
#Choose relevant columns
df=df[['Rooms', 'Bedrooms','Loc_Paris', 'm2', 'nb_bat', 'nb_good', 'nb_bain', 'Cave', 'balc_terr', 'Rénové', 'Parking', 'Chauffage_Central',
       'Chauffage_Individuel', 'Price_int', 'Last_etage']]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2487 entries, 0 to 2486
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rooms                 2487 non-null   object 
 1   Bedrooms              2487 non-null   object 
 2   Loc_Paris             2487 non-null   int64  
 3   m2                    2487 non-null   int64  
 4   nb_bat                2487 non-null   int64  
 5   nb_good               2487 non-null   int64  
 6   nb_bain               2487 non-null   int64  
 7   Cave                  2487 non-null   int64  
 8   balc_terr             2487 non-null   object 
 9   Rénové                2487 non-null   int64  
 10  Parking               2487 non-null   int64  
 11  Chauffage_Central     2487 non-null   int64  
 12  Chauffage_Individuel  2487 non-null   int64  
 13  Price_int             2487 non-null   float64
 14  Last_etage            2487 non-null   int64  
dtypes: float64(1), int64(

In [7]:
#get dummy data
#CHeck the cardinality of our categorical features to know if we wil one hot encode or label encode
df=pd.get_dummies(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2487 entries, 0 to 2486
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Loc_Paris                   2487 non-null   int64  
 1   m2                          2487 non-null   int64  
 2   nb_bat                      2487 non-null   int64  
 3   nb_good                     2487 non-null   int64  
 4   nb_bain                     2487 non-null   int64  
 5   Cave                        2487 non-null   int64  
 6   Rénové                      2487 non-null   int64  
 7   Parking                     2487 non-null   int64  
 8   Chauffage_Central           2487 non-null   int64  
 9   Chauffage_Individuel        2487 non-null   int64  
 10  Price_int                   2487 non-null   float64
 11  Last_etage                  2487 non-null   int64  
 12  Rooms_1 p.                  2487 non-null   uint8  
 13  Rooms_14 p.                 2487 

In [9]:
df.head()

,Loc_Paris,m2,nb_bat,nb_good,nb_bain,Cave,Rénové,Parking,Chauffage_Central,Chauffage_Individuel,...,Bedrooms_1 p.,Bedrooms_2 p.,Bedrooms_3 p.,Bedrooms_4 p.,Bedrooms_5 p.,Bedrooms_6 p.,balc_terr_0,balc_terr_Balcon,balc_terr_Terrasse,"balc_terr_Terrasse, balcon"
0,19,33,14,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,4,122,4,2,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,15,32,9,8,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,11,67,6,5,1,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
4,20,29,5,4,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
#train test split
from sklearn.model_selection import train_test_split
df.Price_int=np.exp(df.Price_int)
X=df.drop('Price_int',axis=1)
y=df.Price_int
X_train,X_test,y_train,y_test=train_test_split(X,y)
y

0        328600.0
1       1940000.0
2        375000.0
3        745000.0
4        346500.0
          ...    
2482    1920000.0
2483     780000.0
2484    4500000.0
2485    2850000.0
2486     622000.0
Name: Price_int, Length: 2487, dtype: float64

In [11]:
#random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf=RandomForestRegressor()
np.mean(cross_val_score(rf,X_train,y_train,scoring='neg_mean_squared_error',cv=5))

-170001707462.5822

In [100]:
#random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf=RandomForestRegressor()
np.mean(cross_val_score(rf,X_train,y_train,cv=5,scoring="neg_mean_squared_error"))

-192483686671.15222